# Intro to Tokenizers

First let us get an understanding of how each tokenizer works. Much of the inspiration here is thanks to the excellent [HuggingFace course on tokenizers](https://huggingface.co/course/chapter6/1?fw=pt).

We will tokenize the following sentence using a simplified version of the three most common tokenizers:

* BPE
* WordPiece
* Unigram (used in SentencePiece)

In [1]:
text = "Hello world! Hey would you help with my work?"
text = text.lower().replace('!', '').replace('?', '')
text

'hello world hey would you help with my work'

Both BPE and WordPiece start by breaking our text into character-level tokens, but they do differ slightly, BPE is the simplest:

In [2]:
#text = ' ' + text + ' '  # adding padding around the text makes life easier later
bpe_text = list(text)
print(bpe_text)

['h', 'e', 'l', 'l', 'o', ' ', 'w', 'o', 'r', 'l', 'd', ' ', 'h', 'e', 'y', ' ', 'w', 'o', 'u', 'l', 'd', ' ', 'y', 'o', 'u', ' ', 'h', 'e', 'l', 'p', ' ', 'w', 'i', 't', 'h', ' ', 'm', 'y', ' ', 'w', 'o', 'r', 'k']


WordPiece identifies start-of-word tokens the same as BPE, but if a token is preceeded by another (non space character) token, it is a *part-of-word* token and prefixed with `##`.

In [3]:
wordpiece_text = []
for i, char in enumerate(text):
    if i == 0 or text[i-1] == ' ' or char == ' ':
        wordpiece_text.append(char)
    else:
        wordpiece_text.append('##'+char)
print(wordpiece_text)

['h', '##e', '##l', '##l', '##o', ' ', 'w', '##o', '##r', '##l', '##d', ' ', 'h', '##e', '##y', ' ', 'w', '##o', '##u', '##l', '##d', ' ', 'y', '##o', '##u', ' ', 'h', '##e', '##l', '##p', ' ', 'w', '##i', '##t', '##h', ' ', 'm', '##y', ' ', 'w', '##o', '##r', '##k']


After breaking our text into these character-level tokens, we can create our *base vocabulary*, which is the initial starting vocab of our tokenizer. From this initial base vocab, all other tokens will be built by merging 'common' pairs.

In [4]:
bpe_vocab = set(bpe_text)
wordpiece_vocab = set(wordpiece_text)

print(f"{bpe_vocab} (len = {len(bpe_vocab)})\n{wordpiece_vocab} (len = {len(wordpiece_vocab)})")

{'d', 'h', 'o', 'y', 'u', 't', 'l', 'r', ' ', 'k', 'i', 'm', 'e', 'w', 'p'} (len = 15)
{'y', 'h', '##l', '##o', '##p', '##t', '##k', ' ', '##i', '##r', '##h', 'm', '##e', '##d', '##u', '##y', 'w'} (len = 17)


With these vocabs we calculate the frequency of each token pair in the corpus. These frequencies identify which pairs to merge. The formula for deciding varies between BPE and WordPiece. We'll lead with the simpler BPE.

In [5]:
def bpe_freq(corpus):
    freq = {}
    for i in range(1, len(corpus)):
        one = corpus[i-1]
        two = corpus[i]
        if ' ' not in [one, two]:
            pair = (one, two)
            if pair not in freq.keys():
                freq[pair] = 0
            freq[pair] += 1
    return freq

In [6]:
bpe_counts = bpe_freq(bpe_text)
print(bpe_counts)

{('h', 'e'): 3, ('e', 'l'): 2, ('l', 'l'): 1, ('l', 'o'): 1, ('w', 'o'): 3, ('o', 'r'): 2, ('r', 'l'): 1, ('l', 'd'): 2, ('e', 'y'): 1, ('o', 'u'): 2, ('u', 'l'): 1, ('y', 'o'): 1, ('l', 'p'): 1, ('w', 'i'): 1, ('i', 't'): 1, ('t', 'h'): 1, ('m', 'y'): 1, ('r', 'k'): 1}


With BPE all we do now is merge the most common pair, in this case both `'he'` and `'wo'` are tied for first place, we will choose the first that appears.

In [7]:
best = 0
merge = ()
for pair in bpe_counts.keys():
    if bpe_counts[pair] > best:
        merge = list(pair)
        best = bpe_counts[pair]
merge

['h', 'e']

Then we go ahead and merge any instances of this pair in our corpus.

In [8]:
for i in range(1, len(bpe_text)):
    if bpe_text[i-1:i+1] == merge:
        # TODO merge the two in the list
        bpe_text = bpe_text[:i-1] + [''.join(bpe_text[i-1:i+1])] + bpe_text[i+1:]
print(bpe_text)

['he', 'l', 'l', 'o', ' ', 'w', 'o', 'r', 'l', 'd', ' ', 'he', 'y', ' ', 'w', 'o', 'u', 'l', 'd', ' ', 'y', 'o', 'u', ' ', 'he', 'l', 'p', ' ', 'w', 'i', 't', 'h', ' ', 'm', 'y', ' ', 'w', 'o', 'r', 'k']


Then we just repeat until reaching a specific vocab size.

In [9]:
while len(bpe_vocab) <= 25:
    # get frequencies
    bpe_counts = bpe_freq(bpe_text)
    # get most common pair
    best = 0
    merge = ()
    for pair in bpe_counts.keys():
        if bpe_counts[pair] > best:
            merge = list(pair)
            best = bpe_counts[pair]
    # then merge any instances of this pair in the corpus
    for i in range(1, len(bpe_text)):
        if bpe_text[i-1:i+1] == merge:
            # TODO merge the two in the list
            bpe_text = bpe_text[:i-1] + [''.join(bpe_text[i-1:i+1])] + bpe_text[i+1:]
    # and add the new pair to the vocab
    bpe_vocab.add(''.join(merge))
    print(', '.join(bpe_text))
print(f'vocab:\n{bpe_vocab}')

he, l, l, o,  , wo, r, l, d,  , he, y,  , wo, u, l, d,  , y, o, u,  , he, l, p,  , w, i, t, h,  , m, y,  , wo, r, k
hel, l, o,  , wo, r, l, d,  , he, y,  , wo, u, l, d,  , y, o, u,  , hel, p,  , w, i, t, h,  , m, y,  , wo, r, k
hel, l, o,  , wor, l, d,  , he, y,  , wo, u, l, d,  , y, o, u,  , hel, p,  , w, i, t, h,  , m, y,  , wor, k
hel, l, o,  , wor, ld,  , he, y,  , wo, u, ld,  , y, o, u,  , hel, p,  , w, i, t, h,  , m, y,  , wor, k
hell, o,  , wor, ld,  , he, y,  , wo, u, ld,  , y, o, u,  , hel, p,  , w, i, t, h,  , m, y,  , wor, k
hello,  , wor, ld,  , he, y,  , wo, u, ld,  , y, o, u,  , hel, p,  , w, i, t, h,  , m, y,  , wor, k
hello,  , world,  , he, y,  , wo, u, ld,  , y, o, u,  , hel, p,  , w, i, t, h,  , m, y,  , wor, k
hello,  , world,  , hey,  , wo, u, ld,  , y, o, u,  , hel, p,  , w, i, t, h,  , m, y,  , wor, k
hello,  , world,  , hey,  , wou, ld,  , y, o, u,  , hel, p,  , w, i, t, h,  , m, y,  , wor, k
hello,  , world,  , hey,  , would,  , y, o, u,  , hel, p,  , w, i, t, 

We've build a BPE tokenizer, now let's apply it to a new sentence.

In [16]:
bpe_vocab = sorted(list(bpe_vocab), key=len, reverse=True)
print(bpe_vocab)

['would', 'hello', 'world', 'hell', 'wor', 'wou', 'hel', 'hey', 'yo', 'ld', 'wo', 'y', 'o', 't', 'u', 'r', 'm', 'e', 'w', 'p', 'd', 'h', 'l', ' ', 'k', 'i']


In [35]:
print(', '.join(list("when i last visited the world with my axolotl they all said hello")))

w, h, e, n,  , i,  , l, a, s, t,  , v, i, s, i, t, e, d,  , t, h, e,  , w, o, r, l, d,  , w, i, t, h,  , m, y,  , a, x, o, l, o, t, l,  , t, h, e, y,  , a, l, l,  , s, a, i, d,  , h, e, l, l, o


In [36]:
new_text = list("when i last visited the world with my axolotl they all said hello")

text_len = len(new_text)

for token in bpe_vocab:
    match = list(token)
    token_len = len(match)
    for i in range(len(new_text) - (token_len-1)):
        if new_text[i:i+token_len] == match:
            new_text = new_text[:i] + [token] + new_text[i+token_len:]
    if text_len != len(new_text):
        text_len = len(new_text)
        print(', '.join(new_text))

# do a final check and replace unknown tokens with '<UNK>'
for i in range(len(new_text)):
    if new_text[i] not in bpe_vocab:
        new_text[i] = '<UNK>'

print(new_text)

w, h, e, n,  , i,  , l, a, s, t,  , v, i, s, i, t, e, d,  , t, h, e,  , w, o, r, l, d,  , w, i, t, h,  , m, y,  , a, x, o, l, o, t, l,  , t, h, e, y,  , a, l, l,  , s, a, i, d,  , hello
w, h, e, n,  , i,  , l, a, s, t,  , v, i, s, i, t, e, d,  , t, h, e,  , world,  , w, i, t, h,  , m, y,  , a, x, o, l, o, t, l,  , t, h, e, y,  , a, l, l,  , s, a, i, d,  , hello
w, h, e, n,  , i,  , l, a, s, t,  , v, i, s, i, t, e, d,  , t, h, e,  , world,  , w, i, t, h,  , m, y,  , a, x, o, l, o, t, l,  , t, hey,  , a, l, l,  , s, a, i, d,  , hello
['w', 'h', 'e', '<UNK>', ' ', 'i', ' ', 'l', '<UNK>', '<UNK>', 't', ' ', '<UNK>', 'i', '<UNK>', 'i', 't', 'e', 'd', ' ', 't', 'h', 'e', ' ', 'world', ' ', 'w', 'i', 't', 'h', ' ', 'm', 'y', ' ', '<UNK>', '<UNK>', 'o', 'l', 'o', 't', 'l', ' ', 't', 'hey', ' ', '<UNK>', 'l', 'l', ' ', '<UNK>', '<UNK>', 'i', 'd', ' ', 'hello']


And that is how BPE tokenizers work!

In [ ]:
bpe_freq = pair_freq(bpe_text)
wordpiece_freq = pair_freq(wordpiece_text)

print(f"{bpe_freq}\n{wordpiece_freq}")

Now we have our counts, we use these frequencies to identify which pairs to merge. To formula for deciding varies between BPE and WordPiece. Again, we'll lead with BPE.

In [55]:
with open('../data/dv-corpus-clean-unique.txt', 'r', encoding='utf-8') as fp:
    dv = fp.read().split('\n')

In [ ]:
len(dv)

['']

In [ ]:
dv[0]

In [44]:
dv = [pair.split('\t')[1] for pair in dv if '\t' in pair]